In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import os

from scipy.optimize import curve_fit
from scipy.stats import chi2 as chi2Dist

from runDataClass import runData
from polyaClass import myPolya
from functionsFIMS import getMagboltzData

In [ ]:
def getGridPixData(runList=None, gridVoltage=None, gainLimit=10):
    """
    TODO
    """
    if runList is None or gridVoltage is None:
        raise ValueError('Error: Invalid inputs.')

    allData = []
    for i, inRun in enumerate(runList):
        simData = runData(inRun)

        rawGain = simData.getCalcParameter('Raw Gain')
        if rawGain >= gainLimit:

            if rawGain <= 100:
                binSize = 1
            if rawGain <= 1000:
                binSize = 10
            if rawGain > 1000:
                binSize = 100

            fitData = simData._fitAvalancheSize(binWidth=binSize)
            fitPolya = fitData['fitPolya']
            
            
            inRunData = {
                'runNo': inRun,
                'gridVoltage': gridVoltage[i],
                'fieldStrength': gridVoltage[i]/5, #for kV/cm
                'rawGain': rawGain,
                'numAvalanche': simData.getRunParameter('Number of Avalanches'),
                'IBN': simData.getCalcParameter('Average IBN'),
                'IBNErr': np.sqrt(simData.getCalcParameter('Average IBN')/simData.getRunParameter('Number of Avalanches')),
                'IBF': simData.getCalcParameter('Average IBF'),
                'IBFErr': simData.getCalcParameter('IBF Error'),
                'IBFGain': simData.getCalcParameter('IBF * Raw Gain'),
                'fitPolyaGain': fitPolya.gain,
                'fitPolyaTheta': fitPolya.theta,
                'fitPolyaGainErr': fitPolya.errors[0],
                'fitPolyaThetaErr': fitPolya.errors[1],
            }
            allData.append(inRunData)

    return pd.DataFrame(allData)

In [ ]:
def chiSquaredAverage(data=None, errors=None):
    """
    TODO
    """

    weightings = 1/errors**2
    average = np.average(data, weights=weightings)

    num = (data - average)**2
    denom = errors**2
    dof = len(data)-1

    chi2 = np.sum(num/denom)
    rchi2 = chi2/dof
    pValue = chi2Dist.sf(chi2, dof)
    
    return average, chi2, rchi2, pValue

In [ ]:
gainLimit = 10

runNoArCO2 = np.arange(21000, 21041)#np.arange(21000, 21048)
voltArCO2 = np.linspace(0, 400, 41)#np.linspace(0, 470, 48)

dataArCO2 = getGridPixData(runList=runNoArCO2, gridVoltage=voltArCO2, gainLimit=gainLimit)
averageThetaArCO2, chi2ThetaArCO2, rchi2ThetaArCO2, pValThetaArCO2 = chiSquaredAverage(dataArCO2['fitPolyaTheta'], dataArCO2['fitPolyaThetaErr'])
averageIBFArCO2, chi2IBFArCO2, rchi2IBFArCO2, pValIBFArCO2 = chiSquaredAverage(dataArCO2['IBF'], dataArCO2['IBFErr'])

runNoT2K = np.arange(21049, 21085)
voltT2K = np.linspace(0, 350, 36)

dataT2K = getGridPixData(runList=runNoT2K, gridVoltage=voltT2K, gainLimit=gainLimit)
averageThetaT2K, chi2ThetaT2K, rchi2ThetaT2K, pValThetaT2K = chiSquaredAverage(dataT2K['fitPolyaTheta'], dataT2K['fitPolyaThetaErr'])
averageIBFT2K, chi2IBFT2K, rchi2IBFT2K, pValIBFT2K = chiSquaredAverage(dataT2K['IBF'], dataT2K['IBFErr'])

In [ ]:
print(f'ArCO2 - Theta:\n\tChi2 = {chi2ThetaArCO2:.2f}\n\trChi2 = {rchi2ThetaArCO2:.2f}\n\tpValue = {pValThetaArCO2:.1e}')
print(f'ArCO2 - IBF:\n\tChi2 = {chi2IBFArCO2:.2f}\n\trChi2 = {rchi2IBFArCO2:.2f}\n\tpValue = {pValIBFArCO2:.1e}')
print(f'T2K - Theta:\n\tChi2 = {chi2ThetaT2K:.2f}\n\trChi2 = {rchi2ThetaT2K:.2f}\n\tpValue = {pValThetaT2K:.1e}')
print(f'T2K - IBF:\n\tChi2 = {chi2IBFT2K:.2f}\n\trChi2 = {rchi2IBFT2K:.2f}\n\tpValue = {pValIBFT2K:.1e}')

In [ ]:
##### Plot Gain vs Field Strength #####
fig = plt.figure(figsize=(5, 5))
fig.suptitle(f'GridPix Gain')

ax1 = fig.add_subplot(111)

##### Gain #####
ax1.plot(
    dataArCO2['fieldStrength'], dataArCO2['rawGain'],
    c='g', label='Ar+CO2 (80/20)'
)
ax1.plot(
    dataT2K['fieldStrength'], dataT2K['rawGain'],
    c='r', label='T2K'
)

ax1.fill_between(
    dataArCO2['fieldStrength'], 
    dataArCO2['rawGain']/2, dataArCO2['rawGain']*2,
    #label='Ar+CO2 Error (factor-of-2)', 
    facecolor='g', alpha=0.25
)

ax1.fill_between(
    dataT2K['fieldStrength'], 
    dataT2K['rawGain']/2, dataT2K['rawGain']*2,
    #label='T2K Error (factor-of-2)', 
    facecolor='r', alpha=0.25
)

ax1.set_title('Gas Gain')
ax1.set_yscale('log')
ax1.set_xlabel('Field Strength (kV/cm)')
ax1.set_ylabel('Average Gain')

ax1.legend()
ax1.grid()

plt.tight_layout()
plt.show()

In [ ]:
##### Plot Polya Fit Parameters #####
fig = plt.figure(figsize=(10, 5))
fig.suptitle(f'GridPix Simulations')

ax1 = fig.add_subplot(111)

ax1.errorbar(
    dataArCO2['fitPolyaGain'], dataArCO2['fitPolyaTheta'],
    xerr=dataArCO2['fitPolyaGainErr'], yerr=dataArCO2['fitPolyaThetaErr'],
    ls='', c='g', label='Ar+CO2 (80/20)'
)
ax1.errorbar(
    dataT2K['fitPolyaGain'], dataT2K['fitPolyaTheta'],
    xerr=dataT2K['fitPolyaGainErr'], yerr=dataT2K['fitPolyaThetaErr'],
    ls='', c='r', label='T2K'
)

ax1.axvline(
    x=100, 
    color='m', ls='--', lw=1,
    label='Bin size: 1->10'
)
ax1.axvline(
    x=1000, 
    color='m', ls=':', lw=1,
    label='Bin size: 10->100'
)

ax1.axhline(
    y=averageThetaArCO2, 
    color='g', lw=1,
    label=f'Ar+CO2 (80/20) Average: {averageThetaArCO2:.2f}'
)
ax1.axhline(
    y=averageThetaT2K, 
    color='r', lw=1,
    label=f'T2K Average: {averageThetaT2K:.2f}'
)

ax1.set_title('Polya Fits to Avalanche PDFs')
ax1.set_xscale('log')
ax1.set_xlabel('Polya Mean (Gain)')
ax1.set_ylabel('Polya Shape (Theta)')

ax1.legend()
ax1.grid()

plt.tight_layout()
plt.show()

In [ ]:
##### Plot IBN/IBF vs Gain #####
fig = plt.figure(figsize=(16, 8))
fig.suptitle(f'GridPix Simulated Ion Backflow')

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

##### IBN #####
ax1.scatter(
    dataArCO2['rawGain'], dataArCO2['IBN'],
    c='g', label='Ar+CO2 (80/20)'
)
ax1.scatter(
    dataT2K['rawGain'], dataT2K['IBN'],
    c='r', label='T2K'
)

##### IBF #####
ax2.scatter(
    dataArCO2['rawGain'], dataArCO2['IBF'],
    label='Ar+CO2 (80/20)', c='g'
)
ax2.scatter(
    dataT2K['rawGain'], dataT2K['IBF'],
    label='T2K', c='r'
)
ax1.scatter(
    [2e3], [0.01*2e3],
    label='GridPix Guess', c='b'
)
ax2.scatter(
    [2e3], [0.01],
    label='GridPix Guess', c='b'
)

##### Average IBF #####
ax2.axhline(
    y=averageIBFArCO2, 
    color='g', linestyle='-', linewidth=1, label=f'Ar+CO2 Average: {averageIBFArCO2:.3f}'
)
ax2.axhline(
    y=averageIBFT2K, 
    color='r', linestyle='-', linewidth=1, label=f'T2K Average: {averageIBFT2K:.3f}'
)

##### Errors #####
ax1.fill_between(
    dataArCO2['rawGain'], 
    dataArCO2['IBN']-dataArCO2['IBNErr'], dataArCO2['IBN']+dataArCO2['IBNErr'], 
    label='Ar+CO2 Error', facecolor='g', alpha=0.25
)
ax1.fill_between(
    dataT2K['rawGain'], 
    dataT2K['IBN']-dataT2K['IBNErr'], dataT2K['IBN']+dataT2K['IBNErr'], 
    label='T2K Error', facecolor='r', alpha=0.25
)

ax2.fill_between(
    dataArCO2['rawGain'], 
    dataArCO2['IBF']-dataArCO2['IBFErr'], dataArCO2['IBF']+dataArCO2['IBFErr'],  
    label='Ar+CO2 Error', facecolor='g', alpha=0.25
)
ax2.fill_between(
    dataT2K['rawGain'], 
    dataT2K['IBF']-dataT2K['IBFErr'], dataT2K['IBF']+dataT2K['IBFErr'], 
    label='T2K Error', facecolor='r', alpha=0.25
)


ax1.set_title('Ion BackFlow Number')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel('Gain')
ax1.set_ylabel('IBN')

ax2.set_title('Ion BackFlow Fraction')
ax2.set_xscale('log')
ax2.set_xlabel('Gain')
ax2.set_ylabel('IBF')

ax1.set_ylim([1, 1e3])
ax2.set_ylim([0, .2])

ax1.legend()
ax2.legend()

ax1.grid()
ax2.grid()

plt.tight_layout()
plt.show()

In [ ]:
print(dataT2K['rawGain'].tolist())
print(dataT2K['IBN'].tolist())
print(dataT2K['rawGain'].iloc[10])
print(dataT2K['IBN'].iloc[10])

In [ ]:
##### Plot IBN/IBF vs Field strength #####
fig = plt.figure(figsize=(16, 8))
fig.suptitle(f'GridPix Simulated Ion Backflow')

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

##### IBN #####
ax1.scatter(
    dataArCO2['fieldStrength'], dataArCO2['IBN'],
    c='g', label='Ar+CO2 (80/20)'
)
ax1.scatter(
    dataT2K['fieldStrength'], dataT2K['IBN'],
    c='r', label='T2K'
)

##### IBF #####
ax2.scatter(
    dataArCO2['fieldStrength'], dataArCO2['IBF'],
    label='Ar+CO2 (80/20)', c='g'
)
ax2.scatter(
    dataT2K['fieldStrength'], dataT2K['IBF'],
    label='T2K', c='r'
)

##### Average IBF #####
ax2.axhline(
    y=averageIBFArCO2, 
    color='g', linestyle='-', linewidth=1, label=f'Ar+CO2 Average: {averageIBFArCO2:.3f}'
)
ax2.axhline(
    y=averageIBFT2K, 
    color='r', linestyle='-', linewidth=1, label=f'T2K Average: {averageIBFT2K:.3f}'
)

##### Errors #####
ax1.fill_between(
    dataArCO2['fieldStrength'], 
    dataArCO2['IBN']-dataArCO2['IBNErr'], dataArCO2['IBN']+dataArCO2['IBNErr'], 
    label='Ar+CO2 Error', facecolor='g', alpha=0.25
)
ax1.fill_between(
    dataT2K['fieldStrength'], 
    dataT2K['IBN']-dataT2K['IBNErr'], dataT2K['IBN']+dataT2K['IBNErr'], 
    label='T2K Error', facecolor='r', alpha=0.25
)

ax2.fill_between(
    dataArCO2['fieldStrength'], 
    dataArCO2['IBF']-dataArCO2['IBFErr'], dataArCO2['IBF']+dataArCO2['IBFErr'],  
    label='Ar+CO2 Error', facecolor='g', alpha=0.25
)
ax2.fill_between(
    dataT2K['fieldStrength'], 
    dataT2K['IBF']-dataT2K['IBFErr'], dataT2K['IBF']+dataT2K['IBFErr'], 
    label='T2K Error', facecolor='r', alpha=0.25
)


ax1.set_title('Ion BackFlow Number')
ax1.set_yscale('log')
ax1.set_xlabel('Field Strength (kV/cm)')
ax1.set_ylabel('IBN')

ax2.set_title('Ion BackFlow Fraction')
ax2.set_xlabel('Field Strength (kV/cm)')
ax2.set_ylabel('IBF')

ax1.set_ylim([1, 1e3])
ax2.set_ylim([0, .2])

ax1.legend()
ax2.legend()

ax1.grid()
ax2.grid()

plt.tight_layout()
plt.show()

In [ ]:
dataArCO2['driftField'] = 1.
dataT2K['driftField'] = .280
dataArCO2['pitch'] = 55
dataT2K['pitch'] = 55

dataArCO2['fieldRatio'] = dataArCO2['fieldStrength']/dataArCO2['driftField']
dataT2K['fieldRatio'] = dataT2K['fieldStrength']/dataT2K['driftField']

In [ ]:
##### Plot IBN/IBF vs Field Ratio #####
fig = plt.figure(figsize=(16, 8))
fig.suptitle(f'GridPix Simulated Ion Backflow')

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

##### IBN #####
ax1.scatter(
    dataArCO2['fieldRatio'], dataArCO2['IBN'],
    c='g', label='Ar+CO2 (80/20)'
)
ax1.scatter(
    dataT2K['fieldRatio'], dataT2K['IBN'],
    c='r', label='T2K'
)

##### IBF #####
ax2.scatter(
    dataArCO2['fieldRatio'], dataArCO2['IBF'],
    label='Ar+CO2 (80/20)', c='g'
)
ax2.scatter(
    dataT2K['fieldRatio'], dataT2K['IBF'],
    label='T2K', c='r'
)

##### Average IBF #####
ax2.axhline(
    y=averageIBFArCO2, 
    color='g', linestyle='-', linewidth=1, label=f'Ar+CO2 Average: {averageIBFArCO2:.3f}'
)
ax2.axhline(
    y=averageIBFT2K, 
    color='r', linestyle='-', linewidth=1, label=f'T2K Average: {averageIBFT2K:.3f}'
)

##### Errors #####
ax1.fill_between(
    dataArCO2['fieldRatio'], 
    dataArCO2['IBN']-dataArCO2['IBNErr'], dataArCO2['IBN']+dataArCO2['IBNErr'], 
    label='Ar+CO2 Error', facecolor='g', alpha=0.25
)
ax1.fill_between(
    dataT2K['fieldRatio'], 
    dataT2K['IBN']-dataT2K['IBNErr'], dataT2K['IBN']+dataT2K['IBNErr'], 
    label='T2K Error', facecolor='r', alpha=0.25
)

ax2.fill_between(
    dataArCO2['fieldRatio'], 
    dataArCO2['IBF']-dataArCO2['IBFErr'], dataArCO2['IBF']+dataArCO2['IBFErr'],  
    label='Ar+CO2 Error', facecolor='g', alpha=0.25
)
ax2.fill_between(
    dataT2K['fieldRatio'], 
    dataT2K['IBF']-dataT2K['IBFErr'], dataT2K['IBF']+dataT2K['IBFErr'], 
    label='T2K Error', facecolor='r', alpha=0.25
)


ax1.set_title('Ion BackFlow Number')
ax1.set_xlabel('Field Ratio')
ax1.set_ylabel('IBN')

ax2.set_title('Ion BackFlow Fraction')
ax2.set_xlabel('Field Ratio')
ax2.set_ylabel('IBF')

ax1.set_yscale('log')
ax2.set_xscale('log')

ax1.legend()
ax2.legend()

ax1.grid()
ax2.grid()

plt.tight_layout()
plt.show()

In [ ]:
#https://inspirehep.net/files/eadf07ca9667269c0c5391cd73dae682
##Reduced field - V/cm*K/mbar
## E Field = 275 V/cm
## T/p = 0.2850 +/- 0.0025 K/mbar
radermacherScale = 1*1013.25/293.15/1000#/0.2850/1000

#Drift Velocity - Figure 6.8
radermacherField = np.array([14.46, 21.75, 29.05, 36.34, 43.52, 50.82, 58.01, 65.32, 72.52, 79.97, 87.29, 94.49, 101.94])*radermacherScale
radermacherVelocity = [20.72, 33.11, 44.64, 54.87, 63.08, 69.14, 73.46, 76.20, 77.64, 78.21, 78.07, 77.49, 76.20]

#Diffusion Coefficients - Table 5.5 and 5.7
radermacherFieldDiffusion = np.array([78.31])*radermacherScale
radermacherFieldDiffusionErr = np.array([0.68])*radermacherScale
radermacherDiffusionL = [211.15]
radermacherDiffusionLErr = [0.21]
radermacherDiffusionT = [277.27]
radermacherDiffusionTErr = [0.75]

In [ ]:
diffusionArCO2 = getMagboltzData('ArCO2-80-20')
diffusionT2K = getMagboltzData('T2K')

In [ ]:
##### Plot Magboltz Results #####
fig = plt.figure(figsize=(16, 8))
fig.suptitle(f'Magboltz Calculations')

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

plotGasses = [
    {
        'gasData': diffusionArCO2,
        'gasName': 'Ar+CO2 (80/20)',
        'c': 'g'
    },
    {
        'gasData': diffusionT2K,
        'gasName': 'T2K',
        'c': 'r'
    }
]

for inGas in plotGasses:

    ##### Drift Velocity #####
    ax1.plot(
        inGas['gasData']['eField'], inGas['gasData']['driftVelocity'],
        c=inGas['c'], label=inGas['gasName']
    )
    ax1.fill_between(
        inGas['gasData']['eField'], 
        inGas['gasData']['driftVelocity']*(1-inGas['gasData']['driftVelocityErr']/100),
        inGas['gasData']['driftVelocity']*(1+inGas['gasData']['driftVelocityErr']/100),
        facecolor=inGas['c'], alpha=.25
    )

    ##### Transverse #####
    ax2.plot(
        inGas['gasData']['eField'], inGas['gasData']['diffusionTransverse'],
        ls='-', c=inGas['c'], label=inGas['gasName']+' - Transverse'
    )
    ax2.fill_between(
        inGas['gasData']['eField'], 
        inGas['gasData']['diffusionTransverse']*(1-inGas['gasData']['diffusionTransverseErr']/100),
        inGas['gasData']['diffusionTransverse']*(1+inGas['gasData']['diffusionTransverseErr']/100),
        facecolor=inGas['c'], alpha=.25, #label=inGas['gasName']+' - Errors'
    )

    ##### Longitudinal #####
    ax2.plot(
        inGas['gasData']['eField'], inGas['gasData']['diffusionLongitudinal'],
        ls='--', c=inGas['c'], label=inGas['gasName']+' - Longitudinal'
    )
    ax2.fill_between(
        inGas['gasData']['eField'], 
        inGas['gasData']['diffusionLongitudinal']*(1-inGas['gasData']['diffusionLongitudinalErr']/100),
        inGas['gasData']['diffusionLongitudinal']*(1+inGas['gasData']['diffusionLongitudinalErr']/100),
        facecolor=inGas['c'], alpha=.25
    )

##### Radermacher Data #####
ax1.scatter(
    radermacherField, radermacherVelocity,
    marker='+', c='b', label='Radermacher (T2K)'
)
ax2.errorbar(
    radermacherFieldDiffusion, radermacherDiffusionL,
    xerr=radermacherFieldDiffusionErr, yerr=radermacherDiffusionLErr,
    marker='x', c='b', label='Radermacher (T2K) - Longitudinal'
)
ax2.errorbar(
    radermacherFieldDiffusion, radermacherDiffusionT,
    xerr=radermacherFieldDiffusionErr, yerr=radermacherDiffusionTErr,
    marker='+', c='b', label='Radermacher (T2K) - Transverse'
)

for inax in [ax1, ax2]:
    inax.axvline(
        x=1, 
        color='g', linestyle=':', label='Drift Field  - ArCO2'
    )
    inax.axvline(
        x=.28, 
        color='r', linestyle=':', label='Drift Field  - T2K'
    )
    inax.axvspan(
        dataArCO2['fieldStrength'].iloc[0], dataArCO2['fieldStrength'].iloc[-1],
        facecolor='g', alpha=.1, label='Amplification Field - ArCO2'
    )
    inax.axvspan(
        dataT2K['fieldStrength'].iloc[0], dataT2K['fieldStrength'].iloc[-1],
        facecolor='r', alpha=.1, label='Amplification Field - T2K'
    )



ax1.set_title('Electron Drift Velocity')
ax1.set_xlabel('Field Strength (kV/cm)')
ax1.set_ylabel('Drift Velocity (um/ns)')
ax1.set_xscale('log')

ax2.set_title('Diffusion Coefficients')
ax2.set_xlabel('Field Strength (kV/cm)')
ax2.set_ylabel('Diffusion (um/sqrt(cm))')
ax2.set_xscale('log')

ax1.legend()
ax2.legend()
ax1.grid()
ax2.grid()

plt.tight_layout()
plt.show()

In [ ]:
##### Plot IBF vs Diffusion Results #####
fig = plt.figure(figsize=(10, 5))
fig.suptitle(f'GridPix Simulations')

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.errorbar(
    dataArCO2['diffusionLForIBF'], dataArCO2['IBF'], yerr=dataArCO2['IBFErr'],
    c='g', label='Ar+CO2 (80/20)', ls='', marker='+'
)
ax1.errorbar(
    dataT2K['diffusionLForIBF'], dataT2K['IBF'], yerr=dataT2K['IBFErr'],
    c='r', label='T2K', ls='', marker='+'
)

ax2.errorbar(
    dataArCO2['diffusionTForIBF'], dataArCO2['IBF'], yerr=dataArCO2['IBFErr'],
    c='g', label='Ar+CO2 (80/20)', ls='', marker='+'
)
ax2.errorbar(
    dataT2K['diffusionTForIBF'], dataT2K['IBF'], yerr=dataT2K['IBFErr'],
    c='r', label='T2K', ls='', marker='+'
)

ax1.axhline(
    y=averageIBFArCO2, 
    color='g', linestyle='-', linewidth=1, label=f'Ar+CO2 Average: {averageIBFArCO2:.3f}'
)
ax1.axhline(
    y=averageIBFT2K, 
    color='r', linestyle='-', linewidth=1, label=f'T2K Average: {averageIBFT2K:.3f}'
)
ax2.axhline(
    y=averageIBFArCO2, 
    color='g', linestyle='-', linewidth=1, label=f'Ar+CO2 Average: {averageIBFArCO2:.3f}'
)
ax2.axhline(
    y=averageIBFT2K, 
    color='r', linestyle='-', linewidth=1, label=f'T2K Average: {averageIBFT2K:.3f}'
)

ax1.set_title('Longitudinal')
ax1.set_xlabel('Diffusion Coefficient (um/sqrt(cm))')
ax1.set_ylabel('Average IBF')

ax2.set_title('Transverse')
ax2.set_xlabel('Diffusion Coefficient (um/sqrt(cm))')
ax2.set_ylabel('Average IBF')

ax1.legend()
ax2.legend()
ax1.grid()
ax2.grid()

plt.tight_layout()
plt.show()

In [ ]:
dataArCO2['sigmaT'] = dataArCO2['diffusionLForIBF']*np.sqrt(50/1e4)
dataT2K['sigmaT'] = dataT2K['diffusionLForIBF']*np.sqrt(50/1e4)

testDataX = np.array([
    0.156, 0.177, 0.205, 0.232, 0.248, 0.266, 0.284, 0.312, 
    0.352, 0.408, 0.455, 0.519, 0.575, 0.646, 0.724, 0.748
])

testDataY = np.array([
    6.74, 5.74, 4.37, 3.22, 2.58, 2.11, 1.89, 1.69,
    1.40, 1.12, 1.07, 1.01, 1.01, 0.95, 0.95, 0.96
])

plotConfigs = [
    {
        'data': dataArCO2,
        'c': 'g',
        'label': 'Ar+CO2 (80/20)',
        'average': averageIBFArCO2
    },
    {
        'data': dataT2K,
        'c': 'r',
        'label': 'T2K',
        'average': averageIBFT2K
    }
]

##### Plot IBF vs Diffusion #####
fig = plt.figure(figsize=(10, 5))
fig.suptitle(f'GridPix Simulations')

ax = fig.add_subplot(111)


for inConfig in plotConfigs:
    
    ax.errorbar(
        inConfig['data']['sigmaT']/inConfig['data']['pitch'], inConfig['data']['IBF']*inConfig['data']['fieldRatio'], 
        yerr=inConfig['data']['IBFErr']*inConfig['data']['fieldRatio'],
        c=inConfig['c'], label=inConfig['label'], ls='', marker='+'
    )

ax.plot(
    testDataX, testDataY,
    c='b', label='TEST DATA'
)


ax.set_title('TEST')
ax.set_xlabel('Diffusion Sigma / Pitch')
ax.set_ylabel('Average IBF * Field Ratio')

ax.legend()
ax.grid()

plt.tight_layout()
plt.show()